In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
con = duckdb.connect("github_stars.db")

In [ ]:
query = """ 
  select * 
  from github_stars.main.repo_stars_monthly as s 
  join github_stars.main.repo_commits_monthly as r 
  on s.repo_id = r.repo_id 
  and end_date is null 
""" 
df_monthly_stars = con.sql(query).df() 
df_monthly_stars.to_csv("monthly_stars.csv") 

In [ ]:
df_monthly_stars = pd.read_csv("monthly_stars.csv")
df_monthly_stars['date'] = pd.to_datetime(df_monthly_stars['date'])
cumulative_stars = df_monthly_stars.groupby(['repo_id', 'date'])['stars'].sum().groupby(level=0).cumsum()
plt.figure(figsize=(12, 6))
for repo_id, group in df_monthly_stars.groupby('repo_id'):
    plt.plot(group['date'], cumulative_stars[group['repo_id'] == repo_id], label=repo_id)
plt.title('Cumulative Stars per Month for Each Repo')
plt.xlabel('Date')
plt.ylabel('Cumulative Stars')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("cumulative_stars.png")
plt.show()